<a href="https://colab.research.google.com/github/lucianomartinsjr/agricultura4.0/blob/main/Agricultura4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalações

In [ ]:
!pip install -q -U crewai
!pip install -q -U langchain_google_genai
!pip install -q -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 325.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

Importando Crew AI

In [ ]:
import os

from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

my_llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )

**Ferramentas**

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

**Agentes**

In [ ]:
buscador = Agent(
    role = 'Agente de Busca e Consulta',
    goal = 'Buscar conteúdo on line sobre informações sobre o clima e condições específicas para uma cultura',
    backstory = '''
          Você é um especialista em agricultura e meteorologia.
          Quero que forneça informações sobre as condições climáticas atuais e previsões para a cultura de {tipo de cultura} na região de {local}.
          Retorne detalhes sobre temperatura, umidade, e outras condições que possam impactar o desenvolvimento dessa cultura.”
                ''',
    llm = my_llm,
    verbose=True,
    allow_delegate=False,
    tools = [search_tool]
)

In [ ]:
fitopatologista = Agent(
    role = 'Especialista para recomendação para Pragas e Doenças',
    goal = 'Fornecer recomendações para o tratamento de pragas e doenças comuns em uma cultura, considerando o clima e as condições locais.',
    backstory = ''''
        Você é um consultor agrícola especializado em fitopatologia.
        Baseado na cultura de {tipo de cultura} e nas condições de umidade e temperatura informadas ([dados de umidade e temperatura]),
        liste as pragas e doenças mais comuns e recomende tratamentos ou práticas preventivas.
        Considere também condições locais como o tipo de solo e histórico de pragas na área.”
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

In [ ]:
especialista_insumos = Agent(
    role = 'Recomendação de Produtos e Equipamentos Agrícolas',
    goal = 'Recomendar produtos, fertilizantes, e equipamentos agrícolas com base no tipo de cultura e nas condições do solo/clima.',
    backstory = ''''
        Você é um especialista em insumos agrícolas e equipamentos.
        Para a cultura de [tipo de cultura], considerando o clima com temperatura de [temperatura] e umidade de [umidade],
        recomende fertilizantes, pesticidas e maquinários adequados. Inclua detalhes como frequência de uso,
        quantidade recomendada e links para produtos, se possível.
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

In [ ]:
agente_monitoramento = Agent(
    role = 'Agente de Coleta de Dados de Sensores',
    goal = 'Receber dados de sensores (umidade, temperatura, tipo de cultura) e registrar as ações recomendadas com base nas condições.',
    backstory = ''''
        Você é um sistema de monitoramento agrícola responsável por registrar e interpretar dados de sensores em tempo real.
        Receba as seguintes informações: tipo de cultura: {tipo de cultura}, umidade do solo: {umidade}, temperatura: {temperatura}.
        Com base nesses dados, registre a ação recomendada (ex: irrigação, aplicação de pesticida) e indique a próxima ação a ser realizada se as condições continuarem as mesmas.”
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

**Tarefas**

In [ ]:
tarefa_busca = Task(
    description='''
        1. Busque informações sobre as condições climáticas atuais para a cultura {tipo_cultura} na região {local}
        2. Colete dados sobre temperatura ideal, umidade necessária e outras condições específicas
        3. Identifique previsões meteorológicas relevantes para o período
        ''',
    agent=buscador,
    expected_output='Relatório detalhado das condições climáticas e requisitos para a cultura especificada'
)

In [ ]:
tarefa_pragas = Task(
    description='''
        1. Analise os dados climáticos coletados
        2. Identifique possíveis pragas e doenças com base nas condições atuais
        3. Recomende tratamentos preventivos e corretivos específicos
        ''',
    agent=fitopatologista,
    expected_output='Lista de pragas/doenças prováveis e recomendações de tratamento'
)

In [ ]:
tarefa_insumos = Task(
    description='''
        1. Com base nos dados climáticos e recomendações do fitopatologista
        2. Sugira fertilizantes e produtos adequados para a situação
        3. Recomende equipamentos necessários para aplicação
        ''',
    agent=especialista_insumos,
    expected_output='Lista de insumos e equipamentos recomendados com especificações de uso'
)

In [ ]:
tarefa_monitoramento = Task(
    description='''
        1. Registre os dados dos sensores de umidade e temperatura
        2. Compare com os parâmetros ideais para a cultura
        3. Determine ações necessárias com base nas condições atuais
        ''',
    agent=agente_monitoramento,
    expected_output='Registro de dados e recomendações de ações imediatas'
)

**Criando Crew (Equipe)**

In [ ]:
crew = Crew(
    agents=[buscador,fitopatologista,especialista_insumos,agente_monitoramento],
    tasks=[tarefa_busca,tarefa_pragas,tarefa_insumos,tarefa_monitoramento],
    verbose=1,
    process=Process.sequential
)

**Executando**

In [ ]:
tema_de_pesquisa = 'Sandro Moreira, organizador do GDG Rio Verde e Professor da UniRV'

entradas = {"tema": tema_de_pesquisa}

resultado = crew.kickoff(inputs=entradas)